In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv("training_data.csv")
test_data = pd.read_csv("test_data.csv")

In [ ]:
data.info()

In [ ]:
data.describe()

Como as colunas city_name e avg_precipitation possuem apenas 1 valor, podemos retirá-las, já que não vão ter nenhuma influência na previsão

In [ ]:
data.nunique()



data2 = data.drop(['city_name', 'avg_precipitation'], axis=1)
test_data2 = test_data.drop(['city_name', 'avg_precipitation'], axis=1)

In [ ]:
data.head(30)

In [ ]:
data2.isnull().sum()

In [ ]:
sns.heatmap(data2.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
sns.heatmap(test_data2.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
i = 0
data_roads = data2.copy()

#print(data2['affected_roads']).head(30)
all_roads = {}
for road_list in data_roads['affected_roads']:
    if type(road_list) == type(float(i)):
        #print("Iteração ::", i)
        data_roads.at[i, 'affected_roads'] = ","
    else: 
        road_list = str(road_list)
        if road_list != ",":
            roads = road_list.split(",")
            #print(roads)
            dic = {}
            
            for road in roads:
                #print(road)
                if road not in dic and road != "":
                    #print(road)
                    dic[road] = 0
                if road not in all_roads and road != "":
                    all_roads[road] = 0
            #data2['affected_roads'][i] = str(list(dic.keys()))
            data_roads.at[i, 'affected_roads'] = list(dic.keys())
            #print(list(dic.keys()))
    i = i + 1
print("Nº de estradas :: ", len(all_roads))
print("Estradas :: ", list(all_roads.keys()))


i = 0

test_data_roads = test_data2.copy()

#print(data2['affected_roads']).head(30)
all_roads_test = {}
for road_list in test_data_roads['affected_roads']:
    if type(road_list) == type(float(i)):
        #print("Iteração ::", i)
        test_data_roads.at[i, 'affected_roads'] = ","
    else: 
        road_list = str(road_list)
        if road_list != ",":
            roads = road_list.split(",")
            #print(roads)
            dic = {}
            
            for road in roads:
                #print(road)
                if road not in dic and road != "":
                    #print(road)
                    dic[road] = 0
                if road not in all_roads_test and road != "":
                    all_roads_test[road] = 0
            #data2['affected_roads'][i] = str(list(dic.keys()))
            test_data_roads.at[i, 'affected_roads'] = list(dic.keys())
            #print(list(dic.keys()))
    i = i + 1
print("Nº de estradas :: ", len(all_roads_test))
print("Estradas :: ", list(all_roads_test.keys()))



In [ ]:
data_roads.head(30)

In [ ]:
for road in all_roads:
    data_roads[road] = 0

for road in all_roads_test:
    test_data_roads[road] = 0
#data2.head(30)

In [ ]:
test_data_roads.head()

In [ ]:
i = 0
for reg in data_roads:
    list = data_roads.at[i, 'affected_roads']
    for road in list:
        if road != ",":
            data_roads.at[i, road] = 1
    i = i + 1


i = 0
for reg in test_data_roads:
    list = test_data_roads.at[i, 'affected_roads']
    for road in list:
        if road != ",":
            test_data_roads.at[i, road] = 1
    i = i + 1

In [ ]:
print(test_data_roads.columns)

In [ ]:
data3 = data_roads.dropna()
sns.heatmap(data_roads.isnull(), yticklabels=False, cbar=False, cmap='viridis')

Como os valores dos incidentes, a nossa label, são categóricos, estes não poderão ser submetidos à maioria dos modelos de ML disponíveis. Assim, teremos de convertê-los para valores numéricos

In [ ]:
replace_map = {'incidents' : {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High':4}}

data4 = data3.replace(replace_map)
data4.head()

Como podemos ver, agora o tipo do feature Incidents é numérico

In [ ]:
data4.info()

Agora vamos tratar das datas

In [ ]:
data4['record_date'].head()

In [ ]:
data4['record_date'] = pd.to_datetime(data4['record_date'], format = "%Y-%m-%d %H:%M", errors='coerce')
assert data4['record_date'].isnull().sum() == 0, 'missing record date'
data4['record_date'].head()

test_data_roads['record_date'] = pd.to_datetime(test_data_roads['record_date'], format = "%Y-%m-%d %H:%M", errors='coerce')

In [ ]:
data4['record_date_year'] = data4['record_date'].dt.year
data4['record_date_month'] = data4['record_date'].dt.month
data4['record_date_day'] = data4['record_date'].dt.day
data4['record_date_hour'] = data4['record_date'].dt.hour
data4['record_date_minute'] = data4['record_date'].dt.minute

test_data_roads['record_date_year'] = test_data_roads['record_date'].dt.year
test_data_roads['record_date_month'] = test_data_roads['record_date'].dt.month
test_data_roads['record_date_day'] = test_data_roads['record_date'].dt.day
test_data_roads['record_date_hour'] = test_data_roads['record_date'].dt.hour
test_data_roads['record_date_minute'] = test_data_roads['record_date'].dt.minute


data4.head()
test_data_roads.head()

In [ ]:
data4.nunique()

Aqui podemos verificar que o ano dos incidentes, bem como o minuto, são features com um único valor, sendo então essa informação inútil.

In [ ]:
data5 = data4.drop(['record_date_minute','record_date_year', 'record_date', 'affected_roads'], axis = 1)
test_data3 = test_data_roads.drop(['record_date_minute','record_date_year', 'record_date', 'affected_roads'], axis = 1)
print(data5['magnitude_of_delay'].unique())
print(data5['luminosity'].unique())
print(data5['avg_rain'].unique())
#data5.info()

A 'magnitude_of_delay' tem 3 valores possíveis: UNDEFINED, MAJOR e MODERATE <br>
A luminosidade tem 3 valores possíveis: LIGHT, LOW_LIGHT, DARK <br>
A 'avg_rain' tem 4 valores possíveis: Sem Chuva, chuva moderada, chuva fraca, chuva forte <br>
Assim, teremos de transformar estes valores em valores numéricos, fazendo label enconding


In [ ]:
replace_map = { 'magnitude_of_delay' : {'UNDEFINED': 0, 'MODERATE': 1, 'MAJOR': 2}, 
                'luminosity': {'DARK': 0, 'LIGHT': 1, 'LOW_LIGHT': 2},
                'avg_rain': {'Sem Chuva': 0, 'chuva moderada': 1, 'chuva fraca': 2, 'chuva forte': 3}
              }
data6 = data5.replace(replace_map)
test_data4 = test_data3.replace(replace_map)

data6.head()
test_data4.head()

Depois de feita toda a etapa de feature engineering, vamos agora construir um modelo de regressão logística.

In [ ]:
df = data6
df_test = test_data4

X = df.drop('incidents', axis=1)
y = df['incidents']
X_test = df_test

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=2022)

In [ ]:
clf.fit(X, y)

In [ ]:
predictions_dt = clf.predict(X_test)
predictions_dt

In [ ]:
#replace_map = {'incidents' : {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High':4}}

outcome = []
for i in range(len(predictions_dt)):
    if predictions_dt[i] == 0:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'None'})
    elif predictions_dt[i] == 1:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Low'})
    elif predictions_dt[i] == 2:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Medium'})
    elif predictions_dt[i] == 3:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'High'})
    elif predictions_dt[i] == 4:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Very_High'})
    
print(outcome)
od = pd.DataFrame(outcome)
od.to_csv("out_file.csv", index=False,header=True)